In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = InceptionResNetV2(weights='imagenet',include_top=False) 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(98304,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# loading the trained weights
model.load_weights("weight_InceptionResnetV2.hdf5")

In [5]:
# compiling the model
from keras.optimizers import SGD
opt = SGD()
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [7]:
# predictions for test videos
all_videos = pd.read_csv("Test_Data.csv")
test_videos = all_videos['video_name']

In [8]:
def count_frames_manual(video):
	# initialize the total number of frames read
	total = 0

	# loop over the frames of the video
	while True:
		# grab the current frame
		(grabbed, frame) = video.read()
	 
		# check to see if we have reached the end of the
		# video
		if not grabbed:
			break

		# increment the total number of frames read
		total += 1

	# return the total number of frames in the video file
	return total

In [9]:
predict = []
actual = []
fgbg = cv2.createBackgroundSubtractorMOG2()

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    #print(videoFile)
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(10) #frame rate
    x=1
    total = count_frames_manual(cap)
    start_frame = int(3*total/4)
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    #while(cap.isOpened()):
    for i in range(1,total):
        cap.set(1,i)
    #    frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
    #    if (frameId % math.floor(frameRate) == 0):
        fgmask = fgbg.apply(frame)
        if count >= start_frame:
            # storing the frames in a new folder named train_1
            filename ='temp/' + "frame%d.jpg" % count
            cv2.imwrite(filename, fgmask)
        else:
            pass
        count+=1
    #cap.release()

    # reading all the frames from temp folder
    images = glob("temp/*.jpg")

    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(299,299,))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)

    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 8*8*1536)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])
#    print("Video, Actual, Predicted: ")
#    print(test_videos[i], videoFile.split('/')[1].split('_')[1], y.columns.values[s.mode(prediction)[0][0]])
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [12:56<00:00, 17.65s/it]


In [10]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

86.36363636363636

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
cnf_matrix = confusion_matrix(actual, predict)
cnf_matrix

array([[ 8,  5],
       [ 1, 30]], dtype=int64)

In [13]:
from sklearn.metrics import classification_report
print(classification_report(actual, predict, labels=['Legside', 'OffSide']))

              precision    recall  f1-score   support

     Legside       0.89      0.62      0.73        13
     OffSide       0.86      0.97      0.91        31

    accuracy                           0.86        44
   macro avg       0.87      0.79      0.82        44
weighted avg       0.87      0.86      0.86        44



In [14]:
actual_video = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[i]
    actual_video.append(videoFile)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 6284.89it/s]


In [15]:
test_results = pd.DataFrame({'predict':predict, 'actual':actual, 'video':actual_video})
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
predict    44 non-null object
actual     44 non-null object
video      44 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [16]:
test_results['Flag'] = (test_results['predict']==test_results['actual']).astype(int)

In [17]:
test_results

,predict,actual,video,Flag
0,OffSide,OffSide,A_OffSide/A_OffSide_shot16.mp4,1
1,OffSide,OffSide,A_OffSide/A_OffSide_shot97.mp4,1
2,OffSide,OffSide,A_OffSide/A_OffSide_shot165.mp4,1
3,OffSide,OffSide,A_OffSide/A_OffSide_shot66.mp4,1
4,OffSide,OffSide,A_OffSide/A_OffSide_shot152.mp4,1
5,OffSide,OffSide,A_OffSide/A_OffSide_shot12.mp4,1
6,OffSide,OffSide,A_OffSide/A_OffSide_shot20.mp4,1
7,OffSide,OffSide,A_OffSide/A_OffSide_shot62.mp4,1
8,OffSide,OffSide,A_OffSide/A_OffSide_shot82.mp4,1
9,OffSide,OffSide,A_OffSide/A_OffSide_shot158.mp4,1
